In [1]:
# -*- coding: utf-8 -*-
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
%matplotlib inline

CATEGORIES = ['alaska', 'bichons', 'french_bulldog', 'chihuahua', 'golden', 'husky', 'labrador', 'papillon', 'samoyed', 'shepherd',
              'teddy', 'basset_hound_dog', 'bull_terrier_dog', 'chinese_sharpei', 'chow',  'cocker_spaniel', 'corgi_dog', 'dachshund_dog',
              'dalmatian_dog', 'doberman', 'eskimo_dog', 'great_greyhound_dog', 'italian_greyhound', 'japanese_spitz_dog', 'lhasa', 'maltese',
              'miniature_pinscher', 'miniature_schnauzer', 'newfoundland', 'pekingese_dog', 'pomeranian', 'poodle', 'rough_collie_dog',
              'saint_bernard', 'shetland_sheepdog', 'shiba_inu_dog', 'shih_tzu_dog', 'tibetan_mastiff', 'wolf_dog']

SAMPLE_PER_CATEGORY = 350
INPUT_SIZE = 224

Using TensorFlow backend.


In [2]:
# read the image
def read_img(data_dir, filepath, size):
    img = image.load_img(os.path.join(data_dir, filepath), target_size=size)
    img = image.img_to_array(img)
    return img

In [ ]:
from collections import Counter
# read the training data
def get_train_val_data(data_dir, train_path):
    SEED = 1991
    train_dir = os.path.join(data_dir, train_path)

    train = []
    for category_id, category in enumerate(CATEGORIES):
        for file in os.listdir(os.path.join(train_dir, category)):
            train.append([train_path+'/{}/{}'.format(category, file), category_id, category])
    train = pd.DataFrame(train, columns=['file', 'category_id', 'category'])
    # sampling
    train = pd.concat([train[train['category'] == c][:SAMPLE_PER_CATEGORY] for c in CATEGORIES])
    train = train.sample(frac=1)
    train.index = np.arange(len(train))

    print(train)

    # read in the image array
    x_train = np.zeros((len(train), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    for i, file in tqdm(enumerate(train['file'])):
        img = read_img(data_dir, file, (INPUT_SIZE, INPUT_SIZE))
        x = preprocess_input(np.expand_dims(img.copy(), axis=0))
        x_train[i] = x

    # split train and validation
    np.random.seed(seed=SEED)
    rnd = np.random.random(len(train))
    train_idx = rnd < 0.7
    valid_idx = rnd >= 0.7

    Xtr = x_train[train_idx]
    Xv = x_train[valid_idx]
    ytr = train.loc[train_idx, 'category_id'].values
    yv = train.loc[valid_idx, 'category_id'].values

    train_dist = Counter(ytr)
    val_dist = Counter(yv)

    for k in train_dist:
        print('Train # {} has {} samples'.format(k, train_dist[k]))
        print('Val # {} has {} samples'.format(k, val_dist[k]))
        print('------------split line---------------------------')

    # to categorical
    ytr = to_categorical(ytr, num_classes=len(CATEGORIES))
    yv = to_categorical(yv, num_classes=len(CATEGORIES))

    num_classes = len(CATEGORIES)

    return Xtr, ytr, Xv, yv, num_classes

In [4]:
data_dir = './input'
train_path = 'train_aug'

Xtr, ytr, Xv, yv, num_classes = get_train_val_data(data_dir, train_path)
print("Categories:{}".format(num_classes))
print('Xtr.shape:{}, Xv.shape:{}'.format(Xtr.shape, Xv.shape))

0it [00:00, ?it/s]

48it [00:00, 476.74it/s]

                                                    file  category_id  \
0      train_aug/shiba_inu_dog/shiba_inu_dog_0_7457.jpeg           35   
1      train_aug/chinese_sharpei/chinese_sharpei_0_19...           13   
2      train_aug/shetland_sheepdog/shetland_sheepdog_...           34   
3                      train_aug/teddy/teddy_0_3700.jpeg           10   
4      train_aug/great_greyhound_dog/great_greyhound_...           21   
5              train_aug/chihuahua/chihuahua_0_2790.jpeg            3   
6      train_aug/rough_collie_dog/rough_collie_dog_0_...           32   
7      train_aug/shetland_sheepdog/shetland_sheepdog_...           34   
8      train_aug/chinese_sharpei/chinese_sharpei_0_92...           13   
9            train_aug/pomeranian/pomeranian_0_6688.jpeg           30   
10                     train_aug/husky/husky_0_2702.jpeg            5   
11     train_aug/tibetan_mastiff/tibetan_mastiff_0_51...           37   
12                 train_aug/samoyed/samoyed_0_2973

99it [00:00, 490.14it/s]

137it [00:00, 452.80it/s]

...

13553it [00:30, 443.34it/s]

13602it [00:30, 443.29it/s]

13649it [00:30, 443.34it/s]

13650it [00:30, 443.34it/s]

Train # 0 has 250 samples
Val # 0 has 100 samples
------------split line---------------------------
Train # 1 has 255 samples
Val # 1 has 95 samples
------------split line---------------------------
Train # 2 has 235 samples
Val # 2 has 115 samples
------------split line---------------------------
Train # 3 has 240 samples
Val # 3 has 110 samples
------------split line---------------------------
Train # 4 has 236 samples
Val # 4 has 114 samples
------------split line---------------------------
Train # 5 has 246 samples
Val # 5 has 104 samples
------------split line---------------------------
Train # 6 has 249 samples
Val # 6 has 101 samples
------------split line---------------------------
Train # 7 has 247 samples
Val # 7 has 103 samples
------------split line---------------------------
Train # 8 has 249 samples
Val # 8 has 101 samples
------------split line---------------------------
Train # 9 has 246 samples
Val # 9 has 104 samples
------------split line---------------------------
T